# Περιγραφή

Σ' αυτό το notebook θα εξεταστούν ορισμένες **μετρικές αξιολόγησης**. Στην παρούσα φάση, ο σκοπός είναι να υπάρξει μία πρώτη εντύπωση όσον αφορά την **απόδοση** αυτών των μετρικών **σε σχέση με τα δεδομένα**.

Είναι χρήσιμο να καταλήξουμε σε μια **μοναδική** μετρική, η οποία θα αποδίδει σχετικά καλά. Σύμφωνα με αυτή θα αξιολογούνται οι επόμενες κινήσεις.

# Βιβλιοθήκες

In [18]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# Δεδομένα

Παρακάτω θα αποδειχτεί ότι τα δεδομένα είναι **imbalanced**. Δηλαδή, τόσο οι υπό εξέταση άνθρωποι όσο και οι δημοσιεύσεις είναι **άνισα** μοιρασμένες στις δύο κατηγορίες.

## Άνθρωποι υπό εξέταση (subjects)

**Αναλογία μεταξύ των Κλάσεων: 7:1**

|            Κατάσταση                | Πλήθος | Ποσοστό |
|:-----------------------------------:|:------:|:-------:|
|  Κινδυνεύουν να αυτο-τραυματιστούν  |   41   |  12.06% |
|      Δεν κινδυνεύουν                |  299   |  87.94% |

In [19]:
txt_path      = '../Data/golden_truth.txt'
df_gt         = pd.read_csv(txt_path, sep=" ", header=None)
df_gt.columns = ['Subject', 'Target']
df_gt['Target'].value_counts()

0    299
1     41
Name: Target, dtype: int64

## Δημοσιεύσεις (posts)
**Αναλογία μεταξύ των Κλάσεων: 24:1**

|         Δημοσιεύσεις        | Πλήθος | Ποσοστό |
|:---------------------------:|:------:|:-------:|
|  Κίνδυνος αυτο-τραυματισμού |   6927 |   4.06% |
|    Δεν υπάρχει κίνδυνος     | 163506 |  95.94% |

In [6]:
csv_path = '../Data/CSVs/first_complete_df.csv'
df_posts = pd.read_csv(csv_path)
df_posts['Target'].value_counts()

0    163506
1      6927
Name: Target, dtype: int64

# Μετρικές

Σ' αυτή τη φάση θα εξεταστούν μόνο οι κοινές μετρικές. Δηλαδή, αυτές που χρησιμοποιούνται πιο συχνά σε προβλήματα κατηγοριοποίησης (classification).

![ConfusionMatrixEl](../Images/confusion_matrix_el.png)
![ConfusionMatrixEng](../Images/confusion_matrix_eng.png)

Στο συγκεκριμένο πρόβλημα, πρέπει να ελαχιστοποιηθούν οι **Ψευδώς Αρνητικές (FN)** προβλέψεις. Γιατί, στην προκειμένη περίπτωση, ένας άνθρωπος που κινδυνεύει να αυτο-τραυματιστεί, θα θεωρηθεί πως δε διατρέχει κανένα κίνδυνο.

* $ Recall = \frac{TP}{TP+FN} $
<br><br>
* $ Percision = \frac{TP}{TP+FP} $
<br><br>
* $ Accuracy = \frac{TP+TN}{TP+TN+FP+FN} $
<br><br>
* $ F1 = 2 \frac{PR}{P+R} $

## Recall (ή Sensitivity)

Δείχνει το ποσοστό των Πραγματικών Θετικών Περιπτώσεων, που προβλέφθηκαν ως τέτοιες.

Έτσι, αν όλα τα δείγματα κατηγοριοποιηθούν στην κλάση «Δεν κινδυνεύει» (Αρνητική Κλάση), τότε $ FN = 0 $. 
   
Άρα $ R = \frac{TP}{TP+FN} = 1 $.

## Precision

Δείχνει το ποσοστό των Προβλεπόμενων Θετικών Περιπτώσεων, που ανήκουν όντως στην κλάση «Κινδυνεύει» (Θετική Κλάση).

## Accuracy

Δείχνει το ποσοστό των προβλέψεων που έγιναν σωστά. Δηλαδή το ποσοστό των δειγμάτων που κατηγοριοποιήθηκαν στη σωστή κλάση.
   
Δεν αποτελεί καλή μετρική σε περίπτωση που τα δεδομένα είναι **ανισομερώς μοιρασμένα (imbalanced)**. Αν όλα τα δείγματα κατηγοριοποιηθούν στην κλάση με τα περισσότερα δείγματα, τότε το accuracy θα τείνει να μεγιστοποιηθεί, χωρίς αυτό να σημαίνει πως το μοντέλο που κατηγοριοποίησε τα δείγματα είναι αποτελεσματικό στην επίλυση του προβλήματος.

Στη συγκεκριμένη περίπτωση, αν κατηγοριοποιούνταν όλα τα δείγματα στην Κλάση "Δεν κινδυνεύει", τότε το **accuracy** θα ήταν εξαιρετικά **υψηλό**, όμως η εν λόγω κατηγοριοποίηση είναι πλήρως **αποτυχημένη**. Μάλιστα, μ' αυτό τον τρόπο **τα FNs θα πάρουν τη μέγιστη τιμή τους**, με τα επακόλουθα που αναφέρθηκαν παραπάνω.  

## F-measure

Συνδυάζει Recall και Precision (τα οποία έχουν μια αντιστρόφως ανάλογη σχέση μεταξύ τους) σε μία μόνο μετρική.
   
$ F = \frac{1}{\alpha\frac{1}{P}+(1-\alpha)\frac{1}{R}} = \frac{(\beta^2+1)PR}{\beta^2P+R} $, όπου $ \beta^2 = \frac{1-\alpha}{\alpha} $

$ F {\tiny \beta=1} = \frac{2PR}{P+R} $

# Υποθέσεις - Δοκιμές

## Σενάριο 1

Θεωρούμε πως όλα τα δείγματα κατηγοριοπούντα στην κλάση «Δεν κινδυνεύει» (Αρνητική Κλάση)

In [27]:
predictions = np.zeros(df_gt.shape[0])
print(classification_report(df_gt['Target'], predictions))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94       299
           1       0.00      0.00      0.00        41

    accuracy                           0.88       340
   macro avg       0.44      0.50      0.47       340
weighted avg       0.77      0.88      0.82       340



## Σενάριο 2

Θεωρούμε πως όλα τα δείγματα κατηγοριοπούνται στην κλάση «Κινδυνεύει» (Θετική Κλάση)

In [66]:
predictions = np.ones(df_gt.shape[0])
print(classification_report(df_gt['Target'], predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       299
           1       0.12      1.00      0.22        41

    accuracy                           0.12       340
   macro avg       0.06      0.50      0.11       340
weighted avg       0.01      0.12      0.03       340



## Σενάριο 3

Τα δείγματα κατηγοριοποιούνται τυχαία σε κάθε κλάση με πιθανότητα 50%

In [67]:
predictions = np.random.randint(0,2,df_gt.shape[0])

ones  = np.count_nonzero(predictions)
zeros = len(predictions) - ones
print("Ones:", ones)
print("Zeros:", zeros)

print(classification_report(df_gt['Target'], predictions))

Ones: 160
Zeros: 180
              precision    recall  f1-score   support

           0       0.88      0.53      0.66       299
           1       0.12      0.46      0.19        41

    accuracy                           0.52       340
   macro avg       0.50      0.50      0.42       340
weighted avg       0.79      0.52      0.60       340



# Πηγές

Manning Ch. D., Raghavan P., Schütze H. (2009) *Introduction to Information Retrieval.* **Evaluation in information retrieval 8**:179-182